# Update Logs
## 2020.12.20
#### Updated terms
- Rename the main program file `bert_BiLSTM-CRF.ipynb` to 
`bertNER.ipynb`.
- 模型儲存與載入路徑一律由變數 `MODEL_SAVE_DIR` 指定
- NERAcceptanceCallback的預設參數 `mode='max'` 改為 `mode='auto'`

#### Pull terms
- bertNER.ipynb
- model_acceptance_callback.py
- albertNER.ipynb & robertaNER.ipynb 如果你需要的話

# Setup the workspace
Here is for google colab.

In [1]:
import os
from google.colab import drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir("/content/drive/MyDrive/AICUP2020/")

# Prepare Environments

In [4]:
!pip install kashgari>=2.0.0 &> /dev/null
from datetime import datetime
from configparser import ConfigParser
import re
import numpy as np
import pandas as pd
import tensorflow as tf

import kashgari
from kashgari.embeddings import BertEmbedding, TransformerEmbedding
from embedding_model import AlbertEmbedding, RoBertaEmbedding
from bilstm_crf_model import BiLSTM_CRF_Model, DoubleBiLSTM_CRF_Model
from bigru_crf_model import BiGRU_CRF_Model
from model_acceptance_callback import NERAcceptanceCallBack

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from utils import load_test_file, format_result, split_chunks, save_model, load_model
from data_generator import read_data, generate_dataset

In [5]:
# Define const
MODEL_SAVE_DIR = 'model/'

# Get config
config = ConfigParser()
config.read('config.ini')
LSTM_UNITS = int(config['MODEL']['lstm_units'])
DROPOUT = float(config['MODEL']['dropout'])
EPOCHS = int(config['TRAIN']['epochs'])
BATCH_SIZE = int(config['TRAIN']['batch_size'])
PREDICT_BATCH_SIZE = int(config['PREDICT']['batch_size'])

# Read Training Data

In [6]:
# x is a list of characters, e.g ['你', '好', '嗎', ...]
# y is a list of labels of each characters, e.g ['O', 'O', 'O', ...]
training_data = read_data(['data/TRAIN_FINAL'], end_flag='')
train_x = list(map(lambda x: [wl[0] for wl in x], training_data))
train_y = list(map(lambda x: [wl[1] for wl in x], training_data))

validation_data = read_data(['data/TEST_FINAL'], end_flag='')
valid_x = list(map(lambda x: [wl[0] for wl in x], validation_data))
valid_y = list(map(lambda x: [wl[1] for wl in x], validation_data))

In [7]:
train_x_chunks = []
for article in train_x:
  article_chunks = split_chunks(article)
  train_x_chunks.extend(article_chunks)

train_y_chunks = []
for article in train_y:
  article_chunks = split_chunks(article)
  train_y_chunks.extend(article_chunks)

valid_x_chunks = []
for article in valid_x:
  article_chunks = split_chunks(article)
  valid_x_chunks.extend(article_chunks)

valid_y_chunks = []
for article in valid_y:
  article_chunks = split_chunks(article)
  valid_y_chunks.extend(article_chunks)

# Build Model

In [8]:
# Bert + BiLSTM-CRF
hyper_parameters = {
  'layer_blstm': {
    'units': LSTM_UNITS,
    'return_sequences': True
  },
  'layer_dropout': {
    'rate': DROPOUT
  },
  'layer_time_distributed': {

  },
  'layer_activation': {
    'activation': 'softmax'
  }
}
bert_embed = BertEmbedding('chinese_L-12_H-768_A-12/')
model = BiLSTM_CRF_Model(bert_embed, hyper_parameters=hyper_parameters)

2020-12-31 13:14:06,103 [DEBUG] kashgari - ------------------------------------------------
2020-12-31 13:14:06,104 [DEBUG] kashgari - Loaded transformer model's vocab
2020-12-31 13:14:06,106 [DEBUG] kashgari - config_path       : chinese_L-12_H-768_A-12/bert_config.json
2020-12-31 13:14:06,108 [DEBUG] kashgari - vocab_path      : chinese_L-12_H-768_A-12/vocab.txt
2020-12-31 13:14:06,109 [DEBUG] kashgari - checkpoint_path : chinese_L-12_H-768_A-12/bert_model.ckpt
2020-12-31 13:14:06,110 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused

# Load Model

In [ ]:
# load model
# model, model_info = load_model(f'{MODEL_SAVE_DIR}/', BiLSTM_CRF_Model)
# model.crf_layer = model.layer_crf
# model.compile_model()
# if model_info is not None:
#   start_epoch = model_info['epoch']
#   monitor = model_info['monitor']
#   threshold = model_info['best']
# else:
#   start_epoch = 0
#   threshold = -np.Inf

# print(f'epoch: {start_epoch}\nmonitor: {monitor}\nbest: {threshold}')

# Train Model

In [12]:
# for loaded model
# ner_cb = NERAcceptanceCallBack(f'{MODEL_SAVE_DIR}/', 
#                 kash_model=model, 
#                 validate_data_x=valid_x_chunks, 
#                 validate_data_y=valid_y_chunks,
#                 monitor='f1-score',
#                 threshold=threshold,
#                 history_record_mode='keep')
# history = model.fit(train_x_chunks,
#            train_y_chunks,
#            x_validate=valid_x_chunks,
#            y_validate=valid_y_chunks,
#            epochs=start_epoch + EPOCHS,
#            batch_size=BATCH_SIZE,
#            callbacks=[ner_cb], 
#            fit_kwargs={'initial_epoch': start_epoch})

# for initial-built model
ner_cb = NERAcceptanceCallBack(f'{MODEL_SAVE_DIR}/', 
                kash_model=model,
                validate_data_x=valid_x_chunks,
                validate_data_y=valid_y_chunks,
                monitor='f1-score',
                history_record_mode='new')
history = model.fit(train_x_chunks,
           train_y_chunks,
           x_validate=valid_x_chunks,
           y_validate=valid_y_chunks,
           epochs=10,
           batch_size=BATCH_SIZE,
           callbacks=[ner_cb])

Preparing text vocab dict: 100%|██████████| 200/200 [00:00<00:00, 9175.90it/s]
2020-12-31 13:14:49,203 [DEBUG] kashgari - --- Build vocab dict finished, Total: 1648 ---
2020-12-31 13:14:49,204 [DEBUG] kashgari - Top-10: ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '：', '。', '，', '師', '是', '民']
Preparing text vocab dict: 100%|██████████| 200/200 [00:00<00:00, 14002.95it/s]
2020-12-31 13:14:49,274 [DEBUG] kashgari - --- Build vocab dict finished, Total: 28 ---
2020-12-31 13:14:49,275 [DEBUG] kashgari - Top-10: ['[PAD]', 'O', 'I-time', 'B-time', 'I-med_exam', 'I-name', 'B-med_exam', 'I-location', 'I-money', 'B-name']
Calculating sequence length: 100%|██████████| 200/200 [00:00<00:00, 93403.94it/s]
2020-12-31 13:15:36,561 [DEBUG] kashgari - Calculated sequence length = 511
2020-12-31 13:15:38,945 [DEBUG] kashgari - fit input shape: (2, 128, 511)
2020-12-31 13:15:38,946 [DEBUG] kashgari - fit input shape: (128, 511)


Epoch 1/10
5/5 [==============================] - 71s 10s/step - loss: 750.1139 - accuracy: 0.5566 - val_loss: 1019.9357 - val_accuracy: 0.9754


2020-12-31 13:16:49,740 [DEBUG] kashgari - predict seq_length: None, input: (2, 200, 512)




13/13 [==============================] - 18s 982ms/step


2020-12-31 13:17:07,653 [DEBUG] kashgari - predict output: (200, 512)
2020-12-31 13:17:07,654 [DEBUG] kashgari - predict output argmax: [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]
2020-12-31 13:17:08,366 [INFO] kashgari - Successfully recorded this epoch into model///history/history_0.csv
2020-12-31 13:17:08,367 [INFO] kashgari - Epoch 1: f1-score improved from -inf to 0.00000, saving model to model//


...

2020-12-31 13:25:16,697 [DEBUG] kashgari - predict seq_length: None, input: (2, 200, 512)




13/13 [==============================] - 12s 945ms/step


2020-12-31 13:25:29,171 [DEBUG] kashgari - predict output: (200, 512)
2020-12-31 13:25:29,172 [DEBUG] kashgari - predict output argmax: [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 0 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 2 1 1]
 [0 1 1 ... 1 1 1]]
2020-12-31 13:25:29,844 [INFO] kashgari - Successfully recorded this epoch into model///history/history_0.csv
2020-12-31 13:25:29,845 [INFO] kashgari - Epoch 10: f1-score improved from 0.02054 to 0.03365, saving model to model//



                precision    recall  f1-score   support

      location     0.0000    0.0000    0.0000        53
          time     0.0695    0.0400    0.0508       450
       contact     0.0000    0.0000    0.0000        13
      med_exam     0.0000    0.0000    0.0000        84
         money     0.0000    0.0000    0.0000        11
          name     0.0000    0.0000    0.0000        43
        family     0.0000    0.0000    0.0000        11
    profession     0.0000    0.0000    0.0000        11
clinical_event     0.0000    0.0000    0.0000         2
     education     0.0000    0.0000    0.0000         1

     micro avg     0.0641    0.0265    0.0375       679
     macro avg     0.0461    0.0265    0.0337       679

epoch: 9 precision: 0.046059, recall: 0.026510, f1-score: 0.033651


2020-12-31 13:25:34,008 [INFO] kashgari - model saved to /content/drive/My Drive/AICUP2020/model


# Check Performation

In [13]:
perf = pd.read_csv(f'{MODEL_SAVE_DIR}/history/history_0.csv')
perf

,loss,accuracy,val_loss,val_accuracy,precision,recall,f1-score
0,424.972260,0.764347,1019.935730,0.975392,0.000000,0.00000,0.000000
1,125.410912,0.966130,1009.375854,0.972907,0.000000,0.00000,0.000000
2,92.806396,0.966618,1038.002197,0.978500,0.000000,0.00000,0.000000
3,71.716629,0.968056,1047.966187,0.975264,0.000000,0.00000,0.000000
4,71.438614,0.966241,1007.644165,0.974481,0.000000,0.00000,0.000000
5,66.619026,0.966628,1024.445312,0.976342,0.000000,0.00000,0.000000
6,59.601776,0.966547,1052.875977,0.974617,0.000000,0.00000,0.000000
7,52.855679,0.968626,1023.481201,0.976421,0.000000,0.00000,0.000000
8,50.165981,0.969803,1026.013916,0.978058,0.028039,0.01620,0.020536
9,46.823929,0.970348,1041.674316,0.978809,0.046059,0.02651,0.033651


In [14]:
fig = make_subplots(rows=2, cols=2,
           specs=[[{}, {}], [{'colspan': 2}, None]],
           subplot_titles=('Loss and Validate loss', 'Accuracy and Validate accuracy', 'F1 Score'))
fig.add_trace(go.Scatter(y=perf['loss'], name='loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=perf['val_loss'], name='validate loss'), row=1, col=1)

fig.add_trace(go.Scatter(y=perf['accuracy'], name='accuracy'), row=1, col=2)
fig.add_trace(go.Scatter(y=perf['val_accuracy'], name='validate accuracy'), row=1, col=2)

for prop in perf.columns[4:]:
  fig.add_trace(go.Scatter(y=perf[prop], name=prop), row=2, col=1)

fig.update_layout(height=800, width=1000)

fig.show()

# Prediction

In [15]:
for_predict = load_test_file('./data/1214test.txt')

In [16]:
def predict(model: tf.keras.Model, for_predict):
    output_df = pd.DataFrame()
    for article_id in range(len(for_predict)):
      tokenized_text = list(for_predict[article_id])
      batched_text = split_chunks(tokenized_text)
      batched_labels = model.predict(batched_text)
      labels = [l for batch in batched_labels for l in batch]

      entities_result = format_result(tokenized_text, labels)
      df = pd.DataFrame(entities_result)
      df.insert(0, 'article_id', [article_id for _ in range(len(entities_result))])
      output_df = pd.concat([output_df, df], ignore_index=True)
    
    return output_df

In [17]:
ans = predict(model, for_predict)
ans[['article_id', 'start_position', 'end_position']] = ans[['article_id', 'start_position', 'end_position']].astype(int)

2020-12-31 13:25:51,433 [DEBUG] kashgari - predict seq_length: None, input: (2, 1, 509)


1/1 [==============================] - 5s 5s/step


2020-12-31 13:25:56,603 [DEBUG] kashgari - predict output: (1, 509)
2020-12-31 13:25:56,607 [DEBUG] kashgari - predict output argmax: [[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

1/1 [==============================] - 1s 559ms/step


2020-12-31 13:25:57,212 [DEBUG] kashgari - predict output: (3, 512)
2020-12-31 13:25:57,212 [DEBUG] kashgari - predict output argmax: [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 0 1 ... 1 1 1]]
2020-12-31 13:25:57,230 [DEBUG] kashgari - predict seq_length: None, input: (2, 4, 512)


...

2020-12-31 13:27:24,677 [DEBUG] kashgari - predict output: (7, 512)
2020-12-31 13:27:24,679 [DEBUG] kashgari - predict output argmax: [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]
2020-12-31 13:27:24,711 [DEBUG] kashgari - predict seq_length: None, input: (2, 8, 512)


1/1 [==============================] - 1s 724ms/step


2020-12-31 13:27:25,469 [DEBUG] kashgari - predict output: (8, 512)
2020-12-31 13:27:25,470 [DEBUG] kashgari - predict output argmax: [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 0 1 ... 1 1 1]]
2020-12-31 13:27:25,495 [DEBUG] kashgari - predict seq_length: None, input: (2, 4, 512)


1/1 [==============================] - 1s 543ms/step


2020-12-31 13:27:26,070 [DEBUG] kashgari - predict output: (4, 512)
2020-12-31 13:27:26,074 [DEBUG] kashgari - predict output argmax: [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]]


In [18]:
ans

,article_id,start_position,end_position,entity_text,entity_type
0,4,365,368,三個禮,time
1,5,1490,1493,下個月,time
2,6,1468,1471,兩個月,time
3,11,5824,5827,一個月,time
4,15,708,712,兩個禮拜,time
5,17,625,628,三個月,time
6,27,1039,1043,三個禮拜,time
7,38,662,666,一個禮拜,time
8,48,835,838,三個月,time
9,48,1596,1599,三個月,time


In [ ]:
ans.to_csv(f'./output/output.tsv', sep='\t', index=False)